In [376]:
% reset -f
from __future__ import print_function
from __future__ import division
import math
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import pandas as pd
from torch import np # Torch wrapper for Numpy

import os

import torch
from torch.utils.data.dataset import Dataset
from torch.utils.data import DataLoader
from torchvision import transforms
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from sklearn.preprocessing import MultiLabelBinarizer
# http://pytorch.org/tutorials/beginner/transfer_learning_tutorial.html
import logging
handler=logging.basicConfig(level=logging.INFO)
lgr = logging.getLogger(__name__)

## Setting up global variables

In [380]:
DATA_ROOT ='/root/data/amz/'

IMG_PATH = DATA_ROOT + '/train_small/'
IMG_EXT = '.jpg'
TRAIN_DATA = DATA_ROOT + '/train_small.csv'

IMG_EXTENSIONS = ['.png', '.jpg']

try:
    from PIL import Image
except ImportError:
    import Image

In [399]:
class GenericImageDataset(Dataset):    

    def __init__(self, csv_path, img_path, img_ext, transform=None):
        
        lgr.info ("CSV path:" + csv_path)
        lgr.info ("IMG path:" + img_path)
        tmp_df = pd.read_csv(csv_path, header=None)
                        
        self.mlb = MultiLabelBinarizer()
        self.img_path = img_path
        self.img_ext = img_ext
        self.transform = transform

        self.X_train = tmp_df[0]        
#         self.X_train = self.X_train.ix[1:]
        
        self.y_train = self.mlb.fit_transform(tmp_df[1].str.split()).astype(np.float32)         
                
        lgr.info("DF:\n" + str (self.X_train))
        lgr.info ("self.y_train:\n" + str(self.y_train))

    def __getitem__(self, index):
        lgr.info ("get item:" + str(index))
        path=self.img_path + self.X_train[index] + self.img_ext
#         lgr.info (" --- get item path:" + path)
        img = Image.open(path)
#         img = img.convert('RGB')
#         figure = plt.figure()
#         figure.set_size_inches(5, 5)        
        plt.imshow(img)
        if self.transform is not None: # TypeError: batch must contain tensors, numbers, or lists; found <class 'PIL.Image.Image'>
            img = self.transform(img)
#             print (str (type(img))) # <class 'torch.FloatTensor'>
                
        label = torch.from_numpy(self.y_train[index])
        return img, label

    def __len__(self):
        l=len(self.X_train.index)
        lgr.info ("Lenght:" +str(l))
        return (l)               

In [400]:
transformations = transforms.Compose([transforms.ToTensor()])
dset_train = GenericImageDataset(TRAIN_DATA,IMG_PATH,IMG_EXT,transformations)

train_loader = DataLoader(dset_train,
                          batch_size=1,
                          shuffle=True,
                          num_workers=1 # 1 for CUDA
                         # pin_memory=True # CUDA only
                         )

import torchvision

# functions to show an image
def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))

for batch_idx, (data, target) in enumerate(train_loader):
    print (type(data)) # <class 'torch.FloatTensor'>   
    trans = transforms.ToPILImage()
    plt.imshow(trans(data))

INFO:__main__:CSV path:/root/data/amz//train_small.csv
INFO:__main__:IMG path:/root/data/amz//train_small/
INFO:__main__:DF:
0    train_0
1    train_1
2    train_2
3    train_3
4    train_4
5    train_5
Name: 0, dtype: object
INFO:__main__:self.y_train:
[[ 0.  0.  1.  1.  0.]
 [ 1.  1.  0.  1.  1.]
 [ 0.  0.  1.  1.  0.]
 [ 1.  1.  0.  1.  1.]
 [ 0.  0.  1.  1.  0.]
 [ 1.  1.  0.  1.  1.]]
INFO:__main__:Lenght:6
INFO:__main__:get item:0
INFO:__main__:get item:2
INFO:__main__:get item:4


<class 'torch.FloatTensor'>


ValueError: axes don't match array